# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
F. Walter  ->  F. Walter  |  ['F. Walter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 59 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.05948
extracting tarball to tmp_2409.05948...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 116 bibliographic references in tmp_2409.05948/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.06174


...

extracting tarball to tmp_2409.06258...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.06405


extracting tarball to tmp_2409.06405...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2701/1211882699.py:51: LatexWarning: 2409.06405 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.05948-b31b1b.svg)](https://arxiv.org/abs/2409.05948) | **RUBIES: a complete census of the bright and red distant Universe with JWST/NIRSpec**  |
|| <mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-09-11*|
|*Comments*| *21 pages, 16 figures; submitted to A&A*|
|**Abstract**|            We present the Red Unknowns: Bright Infrared Extragalactic Survey (RUBIES), providing JWST/NIRSpec spectroscopy of red sources selected across ~150 arcmin$^2$ from public JWST/NIRCam imaging in the UDS and EGS fields. RUBIES novel observing strategy offers a well-quantified selection function: the survey is optimised to reach high (>70%) completeness for bright and red (F150W-F444W>2) sources that are very rare. To place these rare sources in context, we simultaneously observe a reference sample of the 2<z<7 galaxy population, sampling sources at a rate that is inversely proportional to their number density in the 3D space of F444W magnitude, F150W-F444W colour, and photometric redshift. In total, RUBIES observes ~3000 targets across $1<z_{phot}<10$ with both the PRISM and G395M dispersers, and ~1500 targets at $z_{phot}>3$ using only the G395M disperser. The RUBIES data reveal a highly diverse population of red sources that span a broad redshift range ($z_{spec}\sim1-9$), with photometric redshift scatter and outlier fraction that are 3 times higher than for similarly bright sources that are less red. This diversity is not apparent from the photometric SEDs. Only spectroscopy reveals that the SEDs encompass a mixture of galaxies with dust-obscured star formation, extreme line emission, a lack of star formation indicating early quenching, and luminous active galactic nuclei. As a first demonstration of our broader selection function we compare the stellar masses and rest-frame U-V colours of the red sources and our reference sample: red sources are typically more massive ($M_*\sim10^{10-11.5} M_\odot$) across all redshifts. However, we find that the most massive systems span a wide range in U-V colour. We describe our data reduction procedure and data quality, and publicly release the reduced RUBIES data and vetted spectroscopic redshifts of the first half of the survey through the DJA.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.06174-b31b1b.svg)](https://arxiv.org/abs/2409.06174) | **No Redshift Evolution in the Fe II/Mg II Flux Ratios of Quasars across Cosmic Time**  |
|| D. Jiang, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-09-11*|
|*Comments*| *23 pages, 9 figures; Accepted for publication in ApJ. The online materials are available at this https URL*|
|**Abstract**|            The Fe II/Mg II emission line flux ratio in quasar spectra serves as a proxy for the relative Fe to alpha-element abundances in the broad line regions of quasars. Due to the expected different enrichment timescales of the two elements, they can be used as a cosmic clock in the early Universe. We present a study of the Fe II/Mg II ratios in a sample of luminous quasars exploiting high-quality near-IR spectra taken primarily by the XQR-30 program with VLT XSHOOTER. These quasars have a median bolometric luminosity of log(L_bol[erg s^-1])~47.3 and cover a redshift range of z=6.0-6.6. The median value of the measured Fe II/Mg II ratios is ~7.9 with a normalized median absolute deviation of ~2.2. In order to trace the cosmic evolution of Fe II/Mg II in an unbiased manner, we select two comparison samples of quasars with similar luminosities and high-quality spectra from the literature, one at intermediate redshifts (z=3.5-4.8) and the other at low redshifts (z=1.0-2.0). We perform the same spectral analysis for all these quasars, including the usage of the same iron template, the same spectral fitting method, and the same wavelength fitting windows. We find no significant redshift evolution in the Fe II/Mg II ratio over the wide redshift range from z=1 to 6.6. The result is consistent with previous studies and supports the scenario of a rapid iron enrichment in the vicinity of accreting supermassive black holes at high redshift.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.06258-b31b1b.svg)](https://arxiv.org/abs/2409.06258) | **Volatile-rich Sub-Neptunes as Hydrothermal Worlds: The Case of K2-18 b**  |
|| C. N. Luu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-09-11*|
|*Comments*| *15 pages, 5 figures, 1 table*|
|**Abstract**|            Temperate exoplanets between the sizes of Earth and Neptune, known as "sub-Neptunes", have emerged as intriguing targets for astrobiology. It is unknown whether these planets resemble Earth-like terrestrial worlds with a habitable surface, Neptune-like giant planets with deep atmospheres and no habitable surface, or something exotic in between. Recent JWST transmission spectroscopy observations of the canonical sub-Neptune K2-18 b revealed ~1% CH4, ~1% CO2, and a non-detection of CO in the atmosphere. While previous studies have proposed that the observed atmospheric composition could help constrain the lower atmosphere conditions and determine the interior structure of sub-Neptunes like K2-18 b, the possible interactions between the atmosphere and a hot, supercritical water ocean at its base remain unexplored. In this work, we investigate whether a global supercritical water ocean, resembling a planetary-scale hydrothermal system, can explain these observations on K2-18 b-like sub-Neptunes through equilibrium aqueous geochemical calculations. We find that the observed atmospheric CH4/CO2 ratio implies a minimum ocean temperature of ~715 K, whereas the corresponding CO/CO2 ratio allows ocean temperatures up to ~1060 K. These results indicate that a global supercritical water ocean on K2-18 b is plausible. While life cannot survive in this ocean, this work represents the first step towards understanding how a global supercritical water ocean may influence observable atmospheric characteristics on volatile-rich sub-Neptunes. Future observations with better constrained NH3 and CO mixing ratios could further help distinguish between possible interior compositions of K2-18 b.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.06405-b31b1b.svg)](https://arxiv.org/abs/2409.06405) | **JADES: Measuring reionization properties using Lyman-alpha emission**  |
|| G. C. Jones, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-09-11*|
|*Comments*| *26 pages, 20 figures, submitted to MNRAS*|
|**Abstract**|            Ly$\alpha$ is the transition to the ground state from the first excited state of hydrogen (the most common element). Resonant scattering of this line by neutral hydrogen greatly impedes its emergence from galaxies, so the fraction of galaxies which show Ly$\alpha$ is a tracer of the neutral fraction of the intergalactic medium (IGM), and thus the history of reionization. In previous works, we used early JWST/NIRSpec data from the JWST Advanced Deep Extragalactic Survey (JADES) to classify and characterise Ly$\alpha$ emitting galaxies (LAEs). This survey is now approaching completion, and the current sample is nearly an order of magnitude larger. From a sample of 784 galaxies in JADES at $4.0<z<14.3$, we find evidence for Ly$\alpha$ emission in 145 sources. We reproduce the previously found correlation between Ly$\alpha$ escape fraction (\fesc) - Ly$\alpha$ rest-frame equivalent width (\rew) and the negative correlation between Ly$\alpha$ velocity offset - \fesc. Both \fesc and \rew decrease with redshift ($z\gtrsim5.5$), indicating the progression of reionization on a population scale. Our data are used to demonstrate an increasing IGM transmission of Ly$\alpha$ from $z\sim14-6$. We measure the completeness-corrected fraction of LAEs ($X_{Ly\alpha}$) from $z=4-9.5$. An application of these $X_{Ly\alpha}$ values to the results of cosmological models suggests a high neutral fraction at $z=7$ ($\rm X_{HI}=0.81_{-0.10}^{+0.07}$), likely suggesting the need for models with updated \rew distributions (based on comparison to other works). This large sample of LAEs and the completeness correction we have detailed will be paramount for unbiased population studies of galaxies in the EoR.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.05948/./figures/zmag_f444w_completeness.png', 'tmp_2409.05948/./figures/egs_footprint.png', 'tmp_2409.05948/./figures/uds_footprint_nopoints_v4_new3a.png', 'tmp_2409.05948/./figures/redshift_comp_oiii_v3_3panel.png']
copying  tmp_2409.05948/./figures/zmag_f444w_completeness.png to _build/html/
copying  tmp_2409.05948/./figures/egs_footprint.png to _build/html/
copying  tmp_2409.05948/./figures/uds_footprint_nopoints_v4_new3a.png to _build/html/
copying  tmp_2409.05948/./figures/redshift_comp_oiii_v3_3panel.png to _build/html/
exported in  _build/html/2409.05948.md
    + _build/html/tmp_2409.05948/./figures/zmag_f444w_completeness.png
    + _build/html/tmp_2409.05948/./figures/egs_footprint.png
    + _build/html/tmp_2409.05948/./figures/uds_footprint_nopoints_v4_new3a.png
    + _build/html/tmp_2409.05948/./figures/redshift_comp_oiii_v3_3panel.png
found figures ['tmp_2409.06174/./fitted_spec_xqr30.png', 'tmp_2409.06174/./FeIIMgII_z.png', 'tmp_2409.06174/./fitted_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\Msun}{\rm M_\odot}$
$\newcommand{\zphot}{z_{\rm phot}}$
$\newcommand{\zspec}{z_{\rm prism}}$
$\newcommand{\re}{r_{\rm e}}$
$\newcommand{\rt}{r_{\rm t}}$
$\newcommand{\va}{v_{\rm a}}$
$\newcommand{\vc}{v_{\rm circ}}$
$\newcommand{\vdisp}{\sigma_0}$
$\newcommand{\Mdyn}{M_{\rm dyn}}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mbar}{M_{\rm bar}}$
$\newcommand{\micron}{\rm \mu m}$
$\newcommand{\Oiii}{[O {\sc iii}]\xspace}$
$\newcommand{\Nii}{[N {\sc ii}]\xspace}$
$\newcommand{\Hb}{{\rm H}\beta\xspace}$
$\newcommand{\Ha}{{\rm H}\alpha\xspace}$
$\newcommand{\AG}[1]{{\bf \color{violet}{#1} }}$</div>



<div id="title">

# RUBIES: a complete census of the bright and red distant Universe with JWST/NIRSpec  

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.05948-b31b1b.svg)](https://arxiv.org/abs/2409.05948)<mark>Appeared on: 2024-09-11</mark> -  _21 pages, 16 figures; submitted to A&A_

</div>
<div id="authors">

<mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We present the $*Red Unknowns: Bright Infrared Extragalactic Survey*$ (RUBIES), providing JWST/NIRSpec spectroscopy of red sources selected across $\sim150$ arcmin $^2$ from public JWST/NIRCam imaging in the UDS and EGS fields. RUBIES novel observing strategy offers a well-quantified selection function: the survey is optimised to reach high ( $>70\%$ ) spectroscopic completeness for bright and red ( $\mathrm{F150W-F444W}>2$ ) sources that are very rare. To place these rare sources in context, we simultaneously observe a reference sample of the $2<z<7$ galaxy population, sampling sources at a rate that is inversely proportional to their number density in the 3D parameter space of F444W magnitude, $\mathrm{F150W-F444W}$ colour, and photometric redshift. In total, RUBIES observes $\sim3000$ targets across $1<z_{\rm phot}<10$ with both the PRISM and G395M dispersers, and $\sim1500$ targets at $z_\mathrm{phot}>3$ using only the G395M disperser. The RUBIES data reveal a highly diverse population of red sources that span a broad redshift range ( $z_\mathrm{spec}\sim1-9$ ), with photometric redshift scatter and outlier fraction that are 3 times higher than for similarly bright sources that are less red. This diversity is not apparent from the photometric spectral energy distributions (SEDs). Only spectroscopy reveals that the SEDs encompass a mixture of galaxies with dust-obscured star formation, extreme line emission, a lack of star formation indicating early quenching, and luminous active galactic nuclei. As a first demonstration of our broader selection function we compare the stellar masses and rest-frame $U-V$ colours of the red sources and our reference sample: red sources are typically more massive ( $M_*\sim10^{10-11.5} \Msun$ ) across all redshifts. However, we also find that the most massive systems span a wide range in $U-V$ colour. We describe our data reduction procedure and data quality, and publicly release the reduced RUBIES data and vetted spectroscopic redshifts of the first half of the survey through the DAWN JWST Archive.

</div>

<div id="div_fig1">

<img src="tmp_2409.05948/./figures/zmag_f444w_completeness.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of photometric redshifts and F444W magnitudes of RUBIES targets for the PRISM (top) and G395M (bottom) observations. Colour coding shows the spectroscopic completeness in each bin: on the left this is computed as the fraction of targets in the RUBIES NIRSpec footprint that are observed. On the right this is calculated as the fraction of observed targets from the full parent catalogue (i.e. the total PRIMER and CEERS area, approximately double the area covered by RUBIES). The RUBIES selection function achieves high ($>50\%$) spectroscopic targeting completeness for bright, high-redshift sources, even reaching $>70\%$ in the extremes of the parameter space.  (*fig:zmag_completeness*)

</div>
<div id="div_fig2">

<img src="tmp_2409.05948/./figures/egs_footprint.png" alt="Fig5.1" width="50%"/><img src="tmp_2409.05948/./figures/uds_footprint_nopoints_v4_new3a.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** RUBIES footprint of 18 NIRSpec/MSA pointings in the UDS and EGS fields. Purple pointings correspond to the first half of observations in January-March 2024 and form the focus of the current data release. Background images show the NIRCam F444W image mosaics, primarily constructed from public imaging of the CEERS and PRIMER surveys. For the UDS we also show the outline of the PRIMER MIRI imaging footprint in pink. (*fig:footprint*)

</div>
<div id="div_fig3">

<img src="tmp_2409.05948/./figures/redshift_comp_oiii_v3_3panel.png" alt="Fig11" width="100%"/>

**Figure 11. -** Redshift and wavelength offset between the observed $\Oiii$$\lambda 5008$ emission lines measured from the PRISM and G395M spectra. Taking the G395M spectrum as `truth', we find a systematic offset of $\Delta z\sim 0.0044$ or $\sim0.25$ detector pixel for the PRISM spectrum, which does not appear to depend significantly on wavelength (grey solid lines show the running median). The scatter can be partially explained by the larger uncertainty for fainter emission lines. In addition, the intrashutter position of the source (i.e. the spatial offset in the dispersion direction) also introduces wavelength offsets of up to 1 pixel, if the source is point-like and located at the edge of the shutter. In practice, high-redshift sources are (moderately) spatially extended, resulting in smaller offsets. We indeed find a correlation between the source position in the slit and the wavelength offset. (*fig:wave_offset*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.05948"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# No Redshift Evolution in the $\ion{Fe}{2}$/$\ion{Mg}{2}$ Flux Ratios of Quasars across Cosmic Time

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.06174-b31b1b.svg)](https://arxiv.org/abs/2409.06174)<mark>Appeared on: 2024-09-11</mark> -  _23 pages, 9 figures; Accepted for publication in ApJ. The online materials are available at this https URL_

</div>
<div id="authors">

D. Jiang, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The $\ion{Fe}{2}$ / $\ion{Mg}{2}$ emission line flux ratio in quasar spectra serves as a proxy for the relative Fe to $\alpha$ -element abundances in the broad line regions of quasars. Due to the expected different enrichment timescales of the two elements, they can be used as a cosmic clock in the early Universe. We present a study of the $\ion{Fe}{2}$ / $\ion{Mg}{2}$ ratios in a sample of luminous quasars exploiting high-quality near-IR spectra taken primarily by the XQR-30 program with VLT XSHOOTER. These quasars have a median bolometric luminosity of log( $L_\mathrm{bol}$ [ erg s $^{-1}$ ] ) $\sim47.3$ and cover a redshift range of $z=6.0$ -- $6.6$ . The median value of the measured $\ion{Fe}{2}$ / $\ion{Mg}{2}$ ratios is $\sim7.9$ with a normalized median absolute deviation of $\sim2.2$ . In order to trace the cosmic evolution of $\ion{Fe}{2}$ / $\ion{Mg}{2}$ in an unbiased manner, we select two comparison samples of quasars with similar luminosities and high-quality spectra from the literature, one at intermediate redshifts ( $z=3.5$ -- $4.8$ ) and the other at low redshifts ( $z=1.0$ -- $2.0$ ).We perform the same spectral analysis for all these quasars, including the usage of the same iron template, the same spectral fitting method, and the same wavelength fitting windows.We find no significant redshift evolution in the $\ion{Fe}{2}$ / $\ion{Mg}{2}$ ratio over the wide redshift range from $z=1$ to 6.6. The result is consistent with previous studies and supports the scenario of a rapid iron enrichment in the vicinity of accreting supermassive black holes at high redshift.

</div>

<div id="div_fig1">

<img src="tmp_2409.06174/./fitted_spec_xqr30.png" alt="Fig5" width="100%"/>

**Figure 5. -** Spectral fitting for two representative spectra in the E-XQR-30 sample at $\lambda_{\rm rest}\sim$$2100 - 3100$ Å.
The heavy atmospheric absorption regions at 18,200--19,400 Å are masked out in the figure. In each panel, the color-coded lines show the power-law continuum plus Balmer continuum (PL + BC, orange), the Balmer continuum only (BC, brown), the scaled $\ion${Fe}{2} template from 2006ApJ...650...57($\ion${Fe}{2}, green), and the $\ion${Mg}{2} emission line ($\ion${Mg}{2}, blue). The red line shows the sum of all components (best-fit model). For each spectrum, the residual flux of the best-fit model with the $\pm 1\sigma$ observed errors (grey ranges) is shown in the lower sub-panel.
We also smooth the errors and use the pink curves to indicate their boundaries.
The full list of the spectra with the best-fit model fits are published as online figures, and the captions of the online figures are the same as the caption of this figure (but for different quasars).
 (*fig:fitted_spec_xqr30*)

</div>
<div id="div_fig2">

<img src="tmp_2409.06174/./FeIIMgII_z.png" alt="Fig9" width="100%"/>

**Figure 9. -** $\ion${Fe}{2}/$\ion${Mg}{2} line flux ratios at redshift from $z=0$ to $7$. The top panel shows the observed flux ratios. The middle panel shows the same ratios after correcting the Eddington ratio dependence (see details in Section \ref{subsec:EddRatio_correlation}).
The bottom panel shows the median and $\pm1\sigma$ scatter of each sample. The filled symbols show the samples analyzed in this work (red: E-XQR-30, green: XQ-100, blue: low-redshift SDSS), and the open symbols show the samples in the literature. The different colors and symbols are described in the legend on the top. The sub-panels in the lower right corners of the top and middle panels show the typical measurement errors in the logarithmic scale. The grey contours (with a step of 0.2 dex in the logarithmic scale) and grey dots represent the low-redshift sample of 2017ApJ...834...203. Their median $\ion${Fe}{2}/$\ion${Mg}{2} flux ratios at six redshift bins (a step of $\Delta z=0.15$) are shown as the black crosses. Those with Eddington ratios over 0.9 are shown as the purple dots, and the median of this subsample is shown as the purple cross. The dashed line and its shaded region are the median value of the SDSS sample and its $\pm1\sigma$ range as our baseline at low redshift. The figure shows no apparent evolution of the line flux ratio with redshift.
 (*fig:FeIIMgII_z*)

</div>
<div id="div_fig3">

<img src="tmp_2409.06174/./fitted_spec_sdss.png" alt="Fig7" width="100%"/>

**Figure 7. -** Same as Figure \ref{fig:fitted_spec_xqr30} but for the low-redshift SDSS sample.
The smoothed errors are not shown for SDSS spectra.
The $\ion${Fe}{2}/$\ion${Mg}{2} ratios of SDSS 081946.32+414247.2 (upper panel) and SDSS 113448.31+333105.2 (lower panel) are $4.31_{-0.10}^{+0.09}$ and $3.75_{-0.07}^{+0.06}$, respectively.
 (*fig:fitted_spec_sdss*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.06174"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

178  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
